In [1]:
import os
import json
import torch
from torchvision import transforms
from PIL import Image
from underthesea import word_tokenize
from torchtext.vocab import build_vocab_from_iterator
import numpy as np
from collections import defaultdict
from tqdm import tqdm  # Thêm tqdm cho thanh tiến trình

# Cấu hình đường dẫn
base_dir = os.path.dirname(os.getcwd())
raw_dir = os.path.join(base_dir, "raw")
processed_dir = os.path.join(base_dir, "processed")
data_file = os.path.join(processed_dir, "vqa_data.json")

print(f"Base directory: {base_dir}")
print(f"Raw directory: {raw_dir}")
print(f"Processed directory: {processed_dir}")
print(f"Data file: {data_file}")

c:\Users\NGUYEN\miniconda3\envs\py39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Base directory: c:\Users\NGUYEN\Desktop\VQA-Vegetable-Fruits
Raw directory: c:\Users\NGUYEN\Desktop\VQA-Vegetable-Fruits\raw
Processed directory: c:\Users\NGUYEN\Desktop\VQA-Vegetable-Fruits\processed
Data file: c:\Users\NGUYEN\Desktop\VQA-Vegetable-Fruits\processed\vqa_data.json


## Kiểm tra dữ liệu thô


In [13]:
# Kiểm tra số lượng folder
folders = [f for f in os.listdir(raw_dir) if os.path.isdir(os.path.join(raw_dir, f))]
num_folders = len(folders)
print(f"Tổng số thư mục: {num_folders}")

# Kiểm tra số lượng ảnh trong mỗi folder
all_images = 0
image_counts = {}
for folder in folders:
    folder_path = os.path.join(raw_dir, folder)
    images = [
        f for f in os.listdir(folder_path) if f.endswith((".jpg", ".jpeg", ".png"))
    ]
    image_counts[folder] = len(images)
    all_images += len(images)
    print(f"\t- '{folder}': {len(images)} ảnh")
print(f"Tổng: {all_images} ảnh")

Tổng số thư mục: 92
	- 'almond': 66 ảnh
	- 'annona_muricata': 120 ảnh
	- 'apple': 153 ảnh
	- 'apricot': 41 ảnh
	- 'artocarpus_heterophyllus': 120 ảnh
	- 'avocado': 96 ảnh
	- 'banana': 103 ảnh
	- 'bayberry': 72 ảnh
	- 'bergamot_pear': 57 ảnh
	- 'black_currant': 73 ảnh
	- 'black_grape': 40 ảnh
	- 'blood_orange': 96 ảnh
	- 'blueberry': 90 ảnh
	- 'breadfruit': 48 ảnh
	- 'candied_date': 50 ảnh
	- 'carambola': 118 ảnh
	- 'cashew_nut': 58 ảnh
	- 'cherry': 178 ảnh
	- 'cherry_tomato': 162 ảnh
	- 'Chinese_chestnut': 145 ảnh
	- 'citrus': 32 ảnh
	- 'coconut': 58 ảnh
	- 'crown_pear': 56 ảnh
	- 'Dangshan_Pear': 34 ảnh
	- 'dekopon': 92 ảnh
	- 'diospyros_lotus': 48 ảnh
	- 'durian': 123 ảnh
	- 'fig': 110 ảnh
	- 'flat_peach': 136 ảnh
	- 'gandaria': 108 ảnh
	- 'ginseng_fruit': 54 ảnh
	- 'golden_melon': 59 ảnh
	- 'grape': 179 ảnh
	- 'grapefruit': 82 ảnh
	- 'grape_white': 124 ảnh
	- 'green_apple': 54 ảnh
	- 'green_dates': 65 ảnh
	- 'guava': 149 ảnh
	- 'Hami_melon': 45 ảnh
	- 'hawthorn': 91 ảnh
	- 'hazelnut

In [14]:
# Kiểm tra số lượng câu hỏi và câu trả lời từ file JSON
with open(data_file, "r", encoding="utf-8") as f:
    vqa_data = json.load(f)

num_questions = 0
num_answers = 0
question_set = set()  # Các câu hỏi duy nhất
answer_set = set()  # Các câu trả lời duy nhất

for entry in vqa_data:
    questions = entry.get("questions", [])
    num_questions += len(questions)
    num_answers += len(questions)  # Mỗi câu hỏi có 1 câu trả lời
    for q in questions:
        question_set.add(q["question"])
        answer_set.add(q["correct_answer"])

print(f"Tổng số câu hỏi: {num_questions}")
print(f"Tổng số câu trả lời: {num_answers}")
print(f"Số câu hỏi duy nhất: {len(question_set)}")
print(f"Số câu trả lời duy nhất: {len(answer_set)}")
print("Danh sách câu hỏi duy nhất:", list(question_set)[:10])
print("Danh sách câu trả lời duy nhất:", list(answer_set)[:10])

Tổng số câu hỏi: 34316
Tổng số câu trả lời: 34316
Số câu hỏi duy nhất: 806
Số câu trả lời duy nhất: 265
Danh sách câu hỏi duy nhất: ['nho đen đặt ở đâu?', 'dưa mật có màu gì?', 'Vị trí của nho đỏ là ở đâu?', 'hồng xiêm được đặt ở chỗ nào?', 'Số lượng dứa là bao nhiêu?', 'Bạn thấy nho đỏ ở đâu?', 'Vị trí của chanh xanh là ở đâu?', 'Có bao nhiêu lê vương miện?', 'Màu sắc của táo tàu là gì?', 'Số lượng thanh long là bao nhiêu?']
Danh sách câu trả lời duy nhất: ['mơ', 'ngón tay', '4', 'lá', 'bưởi chùm', 'hồng', 'đỏ', 'trắng tím', 'táo chua', 'đất']


# Tiền xử lý


In [2]:
# Danh sách từ ghép cần giữ nguyên
fruit_names = [
    "hạnh nhân",
    "mãng cầu xiêm",
    "táo",
    "mơ",
    "mít",
    "bơ",
    "chuối",
    "dâu rừng",
    "cam bergamot",
    "lý chua đen",
    "nho đen",
    "cam đỏ",
    "việt quất",
    "sa kê",
    "chà là sấy",
    "khế",
    "hạt điều",
    "anh đào",
    "cà chua bi",
    "hạt dẻ",
    "cam quýt",
    "dừa",
    "lê vương miện",
    "lê đường sơn",
    "cam dekopon",
    "thị",
    "sầu riêng",
    "sung",
    "đào",
    "quả thanh trà",
    "nhân sâm",
    "dưa vàng",
    "nho",
    "nho trắng",
    "bưởi chùm",
    "táo xanh",
    "táo xanh",
    "ổi",
    "dưa hami",
    "táo gai",
    "hạt phỉ",
    "hồ đào",
    "dưa mật",
    "lê housi",
    "đào mọng",
    "táo tàu",
    "kiwi",
    "quất",
    "chanh vàng",
    "chanh xanh",
    "vải",
    "nhãn",
    "tỳ bà",
    "hạt mắc ca",
    "quýt",
    "xoài",
    "măng cụt",
    "dâu tằm",
    "dưa lưới",
    "hồng xiêm",
    "cam rốn",
    "xuân đào",
    "dưa lưới",
    "ô liu",
    "đu đủ",
    "chanh dây",
    "hạt pecan",
    "hồng",
    "dứa",
    "hạt dẻ cười",
    "thanh long",
    "mận",
    "táo chua",
    "lựu",
    "bưởi",
    "quýt ponkan",
    "mận khô",
    "chôm chôm",
    "phúc bồn tử",
    "nho đỏ",
    "da rắn",
    "lê cát",
    "mía",
    "cam đường",
    "mãng cầu",
    "mận",
    "cam ba lá",
    "hạt óc chó",
    "hồng bì",
    "mận đá",
    "hồng táo",
    "củ sắn",
]
special_phrases = ["không rõ", "bao nhiêu"]

## Chuẩn hóa dữ liệu văn bản và tạo ánh xạ

In [3]:
with open(data_file, "r", encoding="utf-8") as f:
    vqa_data = json.load(f)

# Chuẩn hóa về chữ thường và tạo ánh xạ với thanh tiến trình
index_mapping = {"image_indices": {}, "qa_indices": {}}
image_idx = 0
qa_idx = 0

print("Đang chuẩn hóa dữ liệu văn bản...")
for entry in tqdm(vqa_data, desc="Processing entries"):
    image_id = entry["image_id"]
    index_mapping["image_indices"][str(image_idx)] = image_id
    for q in entry["questions"]:
        q["question"] = q["question"].lower()
        q["correct_answer"] = q["correct_answer"].lower()
        index_mapping["qa_indices"][str(qa_idx)] = {
            "image_idx": image_idx,
            "question": q["question"],
            "answer": q["correct_answer"],
        }
        qa_idx += 1
    image_idx += 1

# Lưu file
normalized_data_file = os.path.join(processed_dir, "vqa_data_normalized.json")
with open(normalized_data_file, "w", encoding="utf-8") as f:
    json.dump(vqa_data, f, ensure_ascii=False, indent=4)

mapping_file = os.path.join(processed_dir, "index_mapping.json")
with open(mapping_file, "w", encoding="utf-8") as f:
    json.dump(index_mapping, f, ensure_ascii=False, indent=4)

print(f"Đã chuẩn hóa và lưu: {normalized_data_file}")
print(f"Đã tạo file ánh xạ: {mapping_file}")
print(f"Tổng số ảnh: {len(index_mapping['image_indices'])}")
print(f"Tổng số câu hỏi/đáp án: {len(index_mapping['qa_indices'])}")

Đang chuẩn hóa dữ liệu văn bản...


Processing entries: 100%|██████████| 8579/8579 [00:00<00:00, 198246.53it/s]


Đã chuẩn hóa và lưu: c:\Users\NGUYEN\Desktop\VQA-Vegetable-Fruits\processed\vqa_data_normalized.json
Đã tạo file ánh xạ: c:\Users\NGUYEN\Desktop\VQA-Vegetable-Fruits\processed\index_mapping.json
Tổng số ảnh: 8579
Tổng số câu hỏi/đáp án: 34316


## Tiền xử lý ảnh

In [4]:
image_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)

images = []
image_ids = []

folders = [f for f in os.listdir(raw_dir) if os.path.isdir(os.path.join(raw_dir, f))]
total_images = sum(
    len(
        [
            f
            for f in os.listdir(os.path.join(raw_dir, folder))
            if f.endswith((".jpg", ".jpeg", ".png"))
        ]
    )
    for folder in folders
)

print(f"Đang xử lý {total_images} ảnh...")
with tqdm(total=total_images, desc="Processing images") as pbar:
    for folder in folders:
        folder_path = os.path.join(raw_dir, folder)
        for img_file in os.listdir(folder_path):
            if img_file.endswith((".jpg", ".jpeg", ".png")):
                img_path = os.path.join(folder_path, img_file)
                img = Image.open(img_path).convert("RGB")
                img_tensor = image_transform(img)
                images.append(img_tensor)
                image_ids.append(f"{folder}/{img_file}")
                pbar.update(1)

# Gộp thành tensor lớn
images_tensor = torch.stack(images)  # [8579, 3, 224, 224]
torch.save(
    {"images": images_tensor, "image_ids": image_ids},
    os.path.join(processed_dir, "images.pt"),
)

print(f"Đã xử lý và lưu: images.pt")
print(f"Kích thước tensor ảnh: {images_tensor.shape}")

Đang xử lý 8579 ảnh...


Processing images:   0%|          | 0/8579 [00:00<?, ?it/s]

Processing images: 100%|██████████| 8579/8579 [00:23<00:00, 358.82it/s]


Đã xử lý và lưu: images.pt
Kích thước tensor ảnh: torch.Size([8579, 3, 224, 224])


## Tiền xử lý văn bản (giữ từ ghép)

In [6]:
# Tokenizer
def custom_tokenizer(text):
    tokens = word_tokenize(text.lower())
    result = []
    i = 0
    while i < len(tokens):
        # Kiểm tra từ ghép dài nhất có thể
        found_compound = False
        for length in range(
            min(4, len(tokens) - i), 0, -1
        ):  # Kiểm tra từ 4 từ xuống 1 từ
            compound = " ".join(tokens[i : i + length])
            if compound in fruit_names or compound in special_phrases:
                result.append(compound)
                i += length
                found_compound = True
                break
        if not found_compound:
            result.append(tokens[i])
            i += 1
    return result


# Load dữ liệu
with open(data_file, "r", encoding="utf-8") as f:
    vqa_data = json.load(f)

questions = [q["question"] for entry in vqa_data for q in entry["questions"]]
answers = [q["correct_answer"] for entry in vqa_data for q in entry["questions"]]


# Xây dựng từ điển
def yield_tokens(data_iter):
    for text in data_iter:
        yield custom_tokenizer(text)


vocab = build_vocab_from_iterator(
    yield_tokens(questions + answers), specials=["<pad>", "<unk>"]
)
vocab.set_default_index(vocab["<unk>"])

# Ép buộc thêm các từ ghép bị thiếu
for fruit in fruit_names:
    if fruit not in vocab:
        vocab.append_token(fruit)
for phrase in special_phrases:
    if phrase not in vocab:
        vocab.append_token(phrase)

# Lưu vocab mới
torch.save(vocab, os.path.join(processed_dir, "vocab.pt"))
print(f"Đã cập nhật và lưu vocab.pt mới. Kích thước từ điển: {len(vocab)}")

# Tạo tensor cho questions.pt và answers.pt với thanh tiến trình
question_sequences = []
answer_sequences = []

print("Đang tokenize và padding dữ liệu văn bản...")
for q, a in tqdm(zip(questions, answers), total=len(questions), desc="Tokenizing QA"):
    q_seq = torch.tensor(vocab(custom_tokenizer(q)))
    a_seq = torch.tensor(vocab(custom_tokenizer(a)))
    question_sequences.append(q_seq)
    answer_sequences.append(a_seq)

# Padding
max_q_len = max(len(seq) for seq in question_sequences)
max_a_len = max(len(seq) for seq in answer_sequences)
question_padded = torch.nn.utils.rnn.pad_sequence(
    question_sequences, batch_first=True, padding_value=vocab["<pad>"]
)
answer_padded = torch.nn.utils.rnn.pad_sequence(
    answer_sequences, batch_first=True, padding_value=vocab["<pad>"]
)

question_padded = question_padded[:, :max_q_len]
answer_padded = answer_padded[:, :max_a_len]

# Lưu tensor
torch.save(question_padded, os.path.join(processed_dir, "questions.pt"))
torch.save(answer_padded, os.path.join(processed_dir, "answers.pt"))
print(f"Đã lưu questions.pt: {question_padded.shape}")
print(f"Đã lưu answers.pt: {answer_padded.shape}")

Đã cập nhật và lưu vocab.pt mới. Kích thước từ điển: 264
Đang tokenize và padding dữ liệu văn bản...


Tokenizing QA: 100%|██████████| 34316/34316 [00:16<00:00, 2121.69it/s]


Đã lưu questions.pt: torch.Size([34316, 7])
Đã lưu answers.pt: torch.Size([34316, 3])


## Chia dữ liệu train/val/test

In [12]:
# Tạo danh sách chỉ số ảnh theo thư mục
folder_to_images = defaultdict(list)
for idx, image_id in index_mapping["image_indices"].items():
    folder = image_id.split("/")[0]
    folder_to_images[folder].append(int(idx))

# Chia đều từng thư mục
train_images = []
val_images = []
test_images = []

print("Đang chia dữ liệu train/val/test...")
for folder, images in tqdm(folder_to_images.items(), desc="Splitting folders"):
    n = len(images)
    train_n = int(0.8 * n)  # 80%
    val_n = int(0.1 * n)  # 10%

    # Đảm bảo ít nhất 1 ảnh cho val và test nếu folder có >= 10 ảnh
    if n >= 10:
        if val_n == 0:
            val_n = 1
            train_n = n - 2  # Giảm train để có 1 ảnh cho test

    # Chia images
    folder_train = images[:train_n]
    folder_val = images[train_n : train_n + val_n]
    folder_test = images[train_n + val_n :]

    # Thêm vào danh sách chung
    train_images.extend(folder_train)
    val_images.extend(folder_val)
    test_images.extend(folder_test)

# In thống kê chi tiết
print("\nThống kê chi tiết:")
for folder, images in folder_to_images.items():
    train_count = sum(1 for img in train_images if img in images)
    val_count = sum(1 for img in val_images if img in images)
    test_count = sum(1 for img in test_images if img in images)
    print(
        f"{folder}: Tổng {len(images)} ảnh - Train: {train_count}, Val: {val_count}, Test: {test_count}"
    )

# Kiểm tra tổng số Q&A
total_train_qa = len(train_images) * 4
total_val_qa = len(val_images) * 4
total_test_qa = len(test_images) * 4

print(f"\nTổng kết:")
print(f"Train: {len(train_images)} ảnh ({total_train_qa} Q&A)")
print(f"Val: {len(val_images)} ảnh ({total_val_qa} Q&A)")
print(f"Test: {len(test_images)} ảnh ({total_test_qa} Q&A)")
print(f"Tổng: {len(train_images) + len(val_images) + len(test_images)} ảnh")
print(f"Tổng Q&A: {total_train_qa + total_val_qa + total_test_qa} cặp")

# Lưu chỉ số ảnh
torch.save(torch.tensor(train_images), os.path.join(processed_dir, "train_indices.pt"))
torch.save(torch.tensor(val_images), os.path.join(processed_dir, "val_indices.pt"))
torch.save(torch.tensor(test_images), os.path.join(processed_dir, "test_indices.pt"))

print(f"Lưu chỉ số ảnh thành công!")

Đang chia dữ liệu train/val/test...


Splitting folders: 100%|██████████| 92/92 [00:00<?, ?it/s]


Thống kê chi tiết:
Chinese_chestnut: Tổng 145 ảnh - Train: 116, Val: 14, Test: 15
Dangshan_Pear: Tổng 34 ảnh - Train: 27, Val: 3, Test: 4
Hami_melon: Tổng 45 ảnh - Train: 36, Val: 4, Test: 5
almond: Tổng 66 ảnh - Train: 52, Val: 6, Test: 8
annona_muricata: Tổng 120 ảnh - Train: 96, Val: 12, Test: 12
apple: Tổng 153 ảnh - Train: 122, Val: 15, Test: 16
apricot: Tổng 41 ảnh - Train: 32, Val: 4, Test: 5
artocarpus_heterophyllus: Tổng 120 ảnh - Train: 96, Val: 12, Test: 12
avocado: Tổng 96 ảnh - Train: 76, Val: 9, Test: 11
banana: Tổng 103 ảnh - Train: 82, Val: 10, Test: 11
bayberry: Tổng 72 ảnh - Train: 57, Val: 7, Test: 8
bergamot_pear: Tổng 57 ảnh - Train: 45, Val: 5, Test: 7
black_currant: Tổng 73 ảnh - Train: 58, Val: 7, Test: 8
black_grape: Tổng 40 ảnh - Train: 32, Val: 4, Test: 4
blood_orange: Tổng 96 ảnh - Train: 76, Val: 9, Test: 11
blueberry: Tổng 90 ảnh - Train: 72, Val: 9, Test: 9
durian: Tổng 123 ảnh - Train: 98, Val: 12, Test: 13
breadfruit: Tổng 48 ảnh - Train: 38, Val: 4, T

crown_pear: Tổng 56 ảnh - Train: 44, Val: 5, Test: 7
dekopon: Tổng 92 ảnh - Train: 73, Val: 9, Test: 10
diospyros_lotus: Tổng 48 ảnh - Train: 38, Val: 4, Test: 6
fig: Tổng 110 ảnh - Train: 88, Val: 11, Test: 11
flat_peach: Tổng 136 ảnh - Train: 108, Val: 13, Test: 15
gandaria: Tổng 108 ảnh - Train: 86, Val: 10, Test: 12
ginseng_fruit: Tổng 54 ảnh - Train: 43, Val: 5, Test: 6
golden_melon: Tổng 59 ảnh - Train: 47, Val: 5, Test: 7
grape: Tổng 179 ảnh - Train: 143, Val: 17, Test: 19
grapefruit: Tổng 82 ảnh - Train: 65, Val: 8, Test: 9
grape_white: Tổng 124 ảnh - Train: 99, Val: 12, Test: 13
green_apple: Tổng 54 ảnh - Train: 43, Val: 5, Test: 6
green_dates: Tổng 65 ảnh - Train: 52, Val: 6, Test: 7
guava: Tổng 149 ảnh - Train: 119, Val: 14, Test: 16
hawthorn: Tổng 91 ảnh - Train: 72, Val: 9, Test: 10
hazelnut: Tổng 43 ảnh - Train: 34, Val: 4, Test: 5
hickory: Tổng 75 ảnh - Train: 60, Val: 7, Test: 8
honey_dew_melon: Tổng 56 ảnh - Train: 44, Val: 5, Test: 7
housi_pear: Tổng 64 ảnh - Train: 5

## Kiểm tra dữ liệu đã xử lý

In [9]:
# Load dữ liệu
images_data = torch.load(os.path.join(processed_dir, "images.pt"))
questions = torch.load(os.path.join(processed_dir, "questions.pt"))
answers = torch.load(os.path.join(processed_dir, "answers.pt"))
vocab = torch.load(os.path.join(processed_dir, "vocab.pt"))
train_indices = torch.load(os.path.join(processed_dir, "train_indices.pt"))
val_indices = torch.load(os.path.join(processed_dir, "val_indices.pt"))
test_indices = torch.load(os.path.join(processed_dir, "test_indices.pt"))

# Kiểm tra kích thước dữ liệu
print(f"Kích thước tensor ảnh: {images_data['images'].shape}")
print(f"Kích thước tensor câu hỏi: {questions.shape}")
print(f"Kích thước tensor đáp án: {answers.shape}")
print(f"Kích thước từ điển: {len(vocab)}")
print(f"Số ảnh train: {len(train_indices)}")
print(f"Số ảnh val: {len(val_indices)}")
print(f"Số ảnh test: {len(test_indices)}")
print(
    f"Tổng số ảnh: {len(train_indices) + len(val_indices) + len(test_indices)}"
)

# Kiểm tra vocab
vocab_list = vocab.get_itos()  # Lấy danh sách từ vựng
print("\nDanh sách từ vựng:")
for idx, token in enumerate(vocab_list):
    print(f"{idx}: {token}")

# Kiểm tra từ ghép tên trái cây
missing_fruits = [fruit for fruit in fruit_names if fruit not in vocab_list]
print("\nCác từ ghép tên trái cây không có trong vocab:")
if missing_fruits:
    for fruit in missing_fruits:
        print(f"- {fruit}")
else:
    print("Tất cả 92 tên trái cây đều có trong vocab!")

# Kiểm tra cụm từ đặc biệt
missing_phrases = [phrase for phrase in special_phrases if phrase not in vocab_list]
print("\nCác cụm từ đặc biệt không có trong vocab:")
if missing_phrases:
    for phrase in missing_phrases:
        print(f"- {phrase}")
else:
    print("Tất cả cụm từ đặc biệt đều có trong vocab!")

Kích thước tensor ảnh: torch.Size([8579, 3, 224, 224])
Kích thước tensor câu hỏi: torch.Size([34316, 7])
Kích thước tensor đáp án: torch.Size([34316, 3])
Kích thước từ điển: 264
Số ảnh train: 6826
Số ảnh val: 816
Số ảnh test: 937
Tổng số ảnh: 8579

Danh sách từ vựng:
0: <pad>
1: <unk>
2: ?
3: là
4: gì
5: quả
6: ở
7: bao nhiêu
8: có
9: đâu
10: đây
11: không rõ
12: đặt
13: của
14: màu
15: số lượng
16: màu sắc
17: cây
18: đỏ
19: xanh
20: vị trí
21: vàng
22: nâu
23: 2
24: bàn
25: 1
26: táo
27: này
28: nho
29: 3
30: đu đủ
31: anh đào
32: đào
33: hồng
34: thanh long
35: mận
36: mía
37: ổi
38: kiwi
39: đĩa
40: hạt dẻ
41: nho đỏ
42: bưởi
43: chanh dây
44: xoài
45: được
46: cam
47: tỳ bà
48: chỗ
49: nào
50: hạt pecan
51: măng cụt
52: nhãn
53: xuân đào
54: vải
55: sầu riêng
56: cà chua bi
57: hạt dẻ cười
58: mít
59: tên
60: khế
61: táo tàu
62: mãng cầu xiêm
63: nho trắng
64: mãng cầu
65: sung
66: chôm chôm
67: 4
68: dưa lưới
69: chuối
70: hạt óc chó
71: phúc bồn tử
72: táo xanh
73: tím
74: bơ
75